Идеи:
1. сгруппировать по фильмам
2. расправиться c youre
3. есть ощущение что в диалогах важны местоимения

import libraries

In [28]:
import numpy as np
import pandas as pd
import re
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import pipeline, model_selection, preprocessing, metrics
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ponomarevandrew/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

data reading

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [5]:
train.shape

(36991, 4)

In [10]:
train.head()

,id,movie,dialogue,genres
0,0,0,I thought you were in a meeting--? <BR> I am. ...,"[u'drama', u'romance']"
1,1,1,Are you sure you're okay? You're pale. <BR> I...,[u'drama']
2,2,2,Go on! Get out! <BR> Mom look don't say anythi...,[u'comedy']
3,3,3,I could have lost my fucking hands. <BR> That ...,"[u'mystery', u'thriller']"
4,4,4,Stick with me on this Gloria. I need you... <...,"[u'crime', u'thriller']"


In [12]:
train['genre_list'] = train['genres'].map(lambda x : x[1:-1].split(', '))
genres = set()
for x in train['genre_list']:
    genres |= set(x)

In [22]:
genres

{"u'action'",
 "u'adventure'",
 "u'animation'",
 "u'biography'",
 "u'comedy'",
 "u'crime'",
 "u'drama'",
 "u'family'",
 "u'fantasy'",
 "u'history'",
 "u'horror'",
 "u'music'",
 "u'musical'",
 "u'mystery'",
 "u'romance'",
 "u'sci-fi'",
 "u'sport'",
 "u'thriller'",
 "u'war'",
 "u'western'"}

In [14]:
train.head()

,id,movie,dialogue,genres,genre_list
0,0,0,I thought you were in a meeting--? <BR> I am. ...,"[u'drama', u'romance']","[u'drama', u'romance']"
1,1,1,Are you sure you're okay? You're pale. <BR> I...,[u'drama'],[u'drama']
2,2,2,Go on! Get out! <BR> Mom look don't say anythi...,[u'comedy'],[u'comedy']
3,3,3,I could have lost my fucking hands. <BR> That ...,"[u'mystery', u'thriller']","[u'mystery', u'thriller']"
4,4,4,Stick with me on this Gloria. I need you... <...,"[u'crime', u'thriller']","[u'crime', u'thriller']"


In [20]:
def remove_html_tags(text):
    text = BeautifulSoup(text, 'html.parser').get_text()
    return text

def clean_text(text):
    text = remove_html_tags(text)
    text = re.sub("\'", "", text)
    text = re.sub("[^a-zA-Z]"," ",text)
    text = ' '.join(text.split())
    text = text.lower()
    return text

In [21]:
train['clean_dialogue'] = train['dialogue'].apply(lambda x: clean_text(x))
train.head()

,id,movie,dialogue,genres,genre_list,clean_dialogue
0,0,0,I thought you were in a meeting--? <BR> I am. ...,"[u'drama', u'romance']","[u'drama', u'romance']",i thought you were in a meeting i am with you
1,1,1,Are you sure you're okay? You're pale. <BR> I...,[u'drama'],[u'drama'],are you sure youre okay youre pale i feel like...
2,2,2,Go on! Get out! <BR> Mom look don't say anythi...,[u'comedy'],[u'comedy'],go on get out mom look dont say anything first...
3,3,3,I could have lost my fucking hands. <BR> That ...,"[u'mystery', u'thriller']","[u'mystery', u'thriller']",i could have lost my fucking hands that would ...
4,4,4,Stick with me on this Gloria. I need you... <...,"[u'crime', u'thriller']","[u'crime', u'thriller']",stick with me on this gloria i need you and if...


In [29]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    no_stopword_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopword_text)

train['clean_dialogue'] = train['clean_dialogue'].apply(lambda x: remove_stopwords(x))

In [30]:
train.head()

,id,movie,dialogue,genres,genre_list,clean_dialogue
0,0,0,I thought you were in a meeting--? <BR> I am. ...,"[u'drama', u'romance']","[u'drama', u'romance']",thought meeting
1,1,1,Are you sure you're okay? You're pale. <BR> I...,[u'drama'],[u'drama'],sure youre okay youre pale feel like shit frig...
2,2,2,Go on! Get out! <BR> Mom look don't say anythi...,[u'comedy'],[u'comedy'],go get mom look dont say anything first im pre...
3,3,3,I could have lost my fucking hands. <BR> That ...,"[u'mystery', u'thriller']","[u'mystery', u'thriller']",could lost fucking hands would kept magic circ...
4,4,4,Stick with me on this Gloria. I need you... <...,"[u'crime', u'thriller']","[u'crime', u'thriller']",stick gloria need teeth arent suzies youll giv...


In [35]:
multilabel_binarizer = preprocessing.MultiLabelBinarizer()
multilabel_binarizer.fit(train['genre_list'])

y = multilabel_binarizer.transform(train['genre_list'])

In [218]:
 x_train, x_val, ytrain, yval = model_selection.train_test_split(train['clean_dialogue'], y, test_size=0.2)

In [217]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
xtrain = tfidf_vectorizer.fit_transform(x_train)
xval = tfidf_vectorizer.transform(x_val)

ValueError: Found array with 0 sample(s) (shape=(0, 10000)) while a minimum of 1 is required by the normalize function.

In [41]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(xtrain, ytrain)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [214]:
y_pred = clf.predict(xval)
metrics.f1_score(yval, y_pred, average="samples"), metrics.accuracy_score(yval, y_pred)

/Users/ponomarevandrew/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


(0.52943126162480125, 0.22219218813353156)

In [45]:
pred_prob = clf.predict_proba(xval)

In [215]:
t = 0.3
predp = (pred_prob >= t).astype(int)
metrics.f1_score(yval, predp, average="samples"), metrics.accuracy_score(yval, predp)

(0.64343830247330724, 0.22084065414245169)

In [197]:
def predict(m):
    m = clean_text(m)
    m = remove_stopwords(m)
    m_vec = tfidf_vectorizer.transform([m])
    m_pred = clf.predict_proba(m_vec)
    t_pred = np.array([[(pred_prob >= 0.3).astype(int) for pred_prob in m_pred[0]]])
    return multilabel_binarizer.inverse_transform(t_pred)

In [198]:
predict(test['dialogue'][110])

[("u'action'", "u'comedy'", "u'drama'", "u'romance'")]

In [141]:
test.head()

,id,dialogue
0,0,Boy! Did you see the way Mama whopped that dep...
1,1,"Gordon, the insurance people are balking on th..."
2,2,Very fancy. Did you design the bottle? <BR> W...
3,3,It makes me so mad. Steven Schwimmer ready to ...
4,4,Something ought to loosen him up ... how comes...


In [199]:
result = test.copy()
result['genres'] = result['dialogue'].map(predict)

In [200]:
result.head()

,id,dialogue,genres
0,0,Boy! Did you see the way Mama whopped that dep...,"[(u'crime', u'drama')]"
1,1,"Gordon, the insurance people are balking on th...","[(u'drama',)]"
2,2,Very fancy. Did you design the bottle? <BR> W...,"[(u'drama',)]"
3,3,It makes me so mad. Steven Schwimmer ready to ...,"[(u'drama',)]"
4,4,Something ought to loosen him up ... how comes...,"[(u'action', u'thriller')]"


In [203]:
result['genres'] = result['genres'].map(lambda x : x[0])

In [204]:
result.head()

,id,dialogue,genres
0,0,Boy! Did you see the way Mama whopped that dep...,"(u'crime', u'drama')"
1,1,"Gordon, the insurance people are balking on th...","(u'drama',)"
2,2,Very fancy. Did you design the bottle? <BR> W...,"(u'drama',)"
3,3,It makes me so mad. Steven Schwimmer ready to ...,"(u'drama',)"
4,4,Something ought to loosen him up ... how comes...,"(u'action', u'thriller')"


In [206]:
result['genres'] = result['genres'].map(lambda x : [xi[2:-1] for xi in x] )

In [207]:
result.head()

,id,dialogue,genres
0,0,Boy! Did you see the way Mama whopped that dep...,"[crime, drama]"
1,1,"Gordon, the insurance people are balking on th...",[drama]
2,2,Very fancy. Did you design the bottle? <BR> W...,[drama]
3,3,It makes me so mad. Steven Schwimmer ready to ...,[drama]
4,4,Something ought to loosen him up ... how comes...,"[action, thriller]"


In [208]:
result['genres'] = result['genres'].map(lambda x : " ".join(x))

In [209]:
result.head()

,id,dialogue,genres
0,0,Boy! Did you see the way Mama whopped that dep...,crime drama
1,1,"Gordon, the insurance people are balking on th...",drama
2,2,Very fancy. Did you design the bottle? <BR> W...,drama
3,3,It makes me so mad. Steven Schwimmer ready to ...,drama
4,4,Something ought to loosen him up ... how comes...,action thriller


In [210]:
result.drop(['dialogue'], axis=1, inplace = True)

In [212]:
result.head(40)

,id,genres
0,0,crime drama
1,1,drama
2,2,drama
3,3,drama
4,4,action thriller
5,5,drama romance
6,6,comedy drama romance
7,7,comedy drama romance
8,8,drama
9,9,action drama


In [213]:
result.to_csv('result3_2.csv', index = False, header=True)

In [221]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000)
xtrain = tfidf_vectorizer.fit_transform(train['clean_dialogue'])
clf.fit(xtrain, y)
                                              

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [222]:
predict(test['dialogue'][110])

[("u'action'", "u'comedy'", "u'drama'")]

In [223]:
result = test.copy()
result['genres'] = result['dialogue'].map(predict)
result['genres'] = result['genres'].map(lambda x : x[0])
result['genres'] = result['genres'].map(lambda x : [xi[2:-1] for xi in x] )
result['genres'] = result['genres'].map(lambda x : " ".join(x))
result.drop(['dialogue'], axis=1, inplace = True)
result.to_csv('result3_3.csv', index = False, header=True)